In [1]:
import json
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder

In [2]:
with open('combined_data.json') as f:
    data = json.load(f)

FileNotFoundError: [Errno 2] No such file or directory: 'combined_data.json'

In [ ]:
texts = [item['text'] for item in data]
intents = [item['intent'] for item in data]

In [ ]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)
word_index = tokenizer.word_index
max_length = max(len(seq) for seq in sequences)
padded_sequences = pad_sequences(sequences, maxlen=max_length, padding='post')

In [ ]:
intent_encoder = LabelEncoder()
encoded_intents = intent_encoder.fit_transform(intents)
num_classes_intents = len(intent_encoder.classes_)

In [ ]:
entity_types = set()
for item in data:
    entity_types.update(item['entities'].keys())
entity_types = list(entity_types)

In [ ]:
entity_labels = np.zeros((len(data), max_length, len(entity_types) + 1))  # +1 for 'O' (no entity)

In [ ]:
for i, item in enumerate(data):
    words = item['text'].split()
    sequence = sequences[i]
    for entity, value in item['entities'].items():
        entity_type_idx = entity_types.index(entity)
        value_words = value.split()
        for j in range(len(words)):
            if words[j:j+len(value_words)] == value_words:
                for k in range(len(value_words)):
                    if j + k < max_length:
                        entity_labels[i, j + k, entity_type_idx + 1] = 1  # +1 to account for 'O'
                break

In [ ]:
entity_labels = np.argmax(entity_labels, axis=-1)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_intent_train, y_intent_val, y_entity_train, y_entity_val = train_test_split(
    padded_sequences, encoded_intents, entity_labels, test_size=0.2, random_state=42
)

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense, TimeDistributed, Bidirectional

In [ ]:
# Define model architecture
input_text = Input(shape=(max_length,))
embedding = Embedding(input_dim=len(word_index) + 1, output_dim=128)(input_text)
bi_lstm = Bidirectional(LSTM(units=64, return_sequences=True))(embedding)
lstm = LSTM(units=64, return_sequences=True)(embedding)

In [ ]:
intent_dense = Dense(units=64, activation='relu')(lstm[:, -1, :])
intent_output = Dense(units=num_classes_intents, activation='softmax', name='intent_output')(intent_dense)

In [ ]:
entity_output = TimeDistributed(Dense(units=len(entity_types) + 1, activation='softmax'), name='entity_output')(lstm)

In [ ]:
model = Model(inputs=input_text, outputs=[intent_output, entity_output])
model.compile(optimizer='adam',
              loss={'intent_output': 'sparse_categorical_crossentropy', 'entity_output': 'sparse_categorical_crossentropy'},
              metrics=['accuracy'])

model.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_3 (InputLayer)        [(None, 9)]                  0         []                            
                                                                                                  
 embedding_2 (Embedding)     (None, 9, 128)               14720     ['input_3[0][0]']             
                                                                                                  
 lstm_3 (LSTM)               (None, 9, 64)                49408     ['embedding_2[0][0]']         
                                                                                                  
 tf.__operators__.getitem_2  (None, 64)                   0         ['lstm_3[0][0]']              
  (SlicingOpLambda)                                                                         

In [ ]:
history = model.fit(X_train,
                    {'intent_output': y_intent_train, 'entity_output': y_entity_train},
                    epochs=10,
                    validation_data=(X_val, {'intent_output': y_intent_val, 'entity_output': y_entity_val}))

Epoch 1/10
17/17 [==============================] - 5s 61ms/step - loss: 1.6744 - intent_output_loss: 1.0942 - entity_output_loss: 0.5802 - intent_output_accuracy: 0.3656 - entity_output_accuracy: 0.8556 - val_loss: 1.5340 - val_intent_output_loss: 1.0894 - val_entity_output_loss: 0.4446 - val_intent_output_accuracy: 0.3462 - val_entity_output_accuracy: 0.8547
Epoch 2/10
17/17 [==============================] - 0s 12ms/step - loss: 1.4681 - intent_output_loss: 1.0751 - entity_output_loss: 0.3930 - intent_output_accuracy: 0.4391 - entity_output_accuracy: 0.8635 - val_loss: 1.4208 - val_intent_output_loss: 1.0363 - val_entity_output_loss: 0.3845 - val_intent_output_accuracy: 0.5692 - val_entity_output_accuracy: 0.8547
Epoch 3/10
17/17 [==============================] - 0s 15ms/step - loss: 1.2299 - intent_output_loss: 0.8749 - entity_output_loss: 0.3550 - intent_output_accuracy: 0.5977 - entity_output_accuracy: 0.8640 - val_loss: 0.9556 - val_intent_output_loss: 0.6063 - val_entity_outpu

In [ ]:
# Evaluate the model
loss, intent_loss, entity_loss, intent_acc, entity_acc = model.evaluate(X_val,
                                                                        {'intent_output': y_intent_val, 'entity_output': y_entity_val})

print(f"Loss: {loss}, Intent Accuracy: {intent_acc}, Entity Accuracy: {entity_acc}")

5/5 [==============================] - 0s 12ms/step - loss: 0.1340 - intent_output_loss: 0.0025 - entity_output_loss: 0.1315 - intent_output_accuracy: 1.0000 - entity_output_accuracy: 0.9573
Loss: 0.13399218022823334, Intent Accuracy: 1.0, Entity Accuracy: 0.9572649598121643


In [ ]:
# Save the model
model.save('nlu_model.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
from tensorflow.keras.models import load_model

# Load the model
model = load_model('nlu_model.h5')

# Function to predict intent and entities
def predict_nlu(text):
    sequence = tokenizer.texts_to_sequences([text])
    padded_sequence = pad_sequences(sequence, maxlen=max_length, padding='post')
    intent_pred, entity_pred = model.predict(padded_sequence)

    intent = intent_encoder.inverse_transform([np.argmax(intent_pred)])
    entities = np.argmax(entity_pred, axis=-1)

    entity_list = []
    for i, idx in enumerate(entities[0]):
        if idx > 0:
            entity = entity_types[idx - 1]
            entity_value = text.split()[i]
            entity_list.append((entity, entity_value))

    return intent[0], entity_list

# Example usage
text = "Find hidden gems in Legian region."
intent, entities = predict_nlu(text)
print(f"Intent: {intent}, Entities: {entities}")

1/1 [==============================] - 1s 511ms/step
Intent: FindTouristAttraction, Entities: [('LOCATION', 'Legian')]


In [ ]:
from collections import Counter

intent_counts = Counter([item['intent'] for item in data])
print(intent_counts)

Counter({'FindTouristAttraction': 228, 'FindRestaurant': 212, 'FindHotel': 207})


In [ ]:
import json

# Load data from all three JSON files
with open('hotel.json', 'r') as f:
    hotel_data = json.load(f)

with open('restaurants.json', 'r') as f:
    restaurants_data = json.load(f)

with open('touristattraction.json', 'r') as f:
    touristattraction_data = json.load(f)

# Combine data into a single list
combined_data = hotel_data + restaurants_data + touristattraction_data

# Save combined data to a new JSON file
output_file = 'combined_data.json'
with open(output_file, 'w') as f:
    json.dump(combined_data, f, indent=2)

print("Combined data has been saved to 'combined_data.json'")

Combined data has been saved to 'combined_data.json'
